In [17]:
# from sklearn.datasets import fetch_olivetti_faces
import numpy as np
import cv2
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog
import imutils

face_cascade = cv2.CascadeClassifier('haar/haarcascade_frontalface_default.xml')
global face_cascade
# data_images = fetch_olivetti_faces()
# data_images = data_images['images']


In [1]:
def template_match(img_path, temp_path):
    origin_img = cv2.imread(img_path)
    img = cv2.imread(img_path,0)
    template = cv2.imread(temp_path,0)
    
    (tH, tW) = template.shape[:2]
    w, h = template.shape[::-1]

    # All the 6 methods for comparison in a list
    #methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
#             'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
    meth = 'cv2.TM_SQDIFF_NORMED'

    method = eval(meth)
    found = None
    for scale in np.linspace(0.2, 1.8, 20)[::-1]:

        resized = imutils.resize(img, width = int(img.shape[1] * scale))
        r = img.shape[1] / float(resized.shape[1])

        if resized.shape[0] < tH or resized.shape[1] < tW:
            break

        result = cv2.matchTemplate(resized, template, method)
        (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)

        if (found is None) or (maxVal > found[0]):
            found = (maxVal, maxLoc, r)

    (_, maxLoc, r) = found
    (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
    (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))

    origin_img = cv2.rectangle(origin_img, (startX, startY), (endX, endY), (255, 0, 0), 2)
            
#         # Apply template Matching
#         res = cv2.matchTemplate(img,template,method)
#         min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

#         # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
#         if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
#             top_left = min_loc
#         else:
#             top_left = max_loc
#         bottom_right = (top_left[0] + w, top_left[1] + h)

#         origin_img = cv2.rectangle(origin_img, top_left, bottom_right, (255,0,0), 2)
    
        
    return cv2.cvtColor(origin_img, cv2.COLOR_BGR2RGB)
        

In [19]:
def face_symmetry(face_cropped):
    size = face_cropped.shape
    w = int(size[1]/10)
    if w < 3:
        w = 3
    line = w
    best_central = [w, 1000000]
    while line < size[1] - w:
        tmp = 0
        for i in range(1, w+1):
            for j in range(size[0]):
                tmp += np.abs(int(face_cropped[j][line-i]) - int(face_cropped[j][line+i]))
        if tmp < best_central[1]:
            best_central = [line, tmp]
        line += 2
    w1 = int(w/2)
    if w1 < 3:
        w1 = 3
    best_left = [w1, 1000000]
    best_right = [best_central[0] + w1, 1000000]
    line1 = w1
    line2 = best_central[0] + w1
    while line1 < best_central[0] - w1 and line2 < size[1] - w1:
        tmp1 = 0
        tmp2 = 0
        for i in range(1, w1+1):
            for j in range(int(size[0]/4), int(3*size[0]/4)):
                tmp1 += np.abs(int(face_cropped[j][line1-i]) - int(face_cropped[j][line1+i]))
                tmp2 += np.abs(int(face_cropped[j][line2-i]) - int(face_cropped[j][line2+i]))
        if tmp1 < best_left[1]:
            best_left = [line1, tmp1]
        if tmp2 < best_right[1]:
            best_right = [line2, tmp2]
        line1 += 2
        line2 += 2
    return best_central[0], best_left[0], best_right[0]

In [20]:
def symmetry_lines(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        borders = int(w/10)

        c, l, r = face_symmetry(gray[y:y+h, x+borders:x+w-borders])
        img = cv2.rectangle(img, (x+borders, y), (x + w-borders, y + h), (255, 0, 0), 2)
        img = cv2.line(img, (x+borders+c, y), (x+borders+c, y+h), (0,255,0), 2) 
        img = cv2.line(img, (x+borders+l, y + int(h/4)), (x+borders+l, y + int(3*h/4)), (0,0,255), 2) 
        img = cv2.line(img, (x+borders+r, y + int(h/4)), (x+borders+r, y + int(3*h/4)), (0,0,255), 2) 
#         eyes = self.eye_cascade.detectMultiScale(self.image)
#         for (ex, ey, ew, eh) in eyes[:2]:
#             self.frame = cv2.rectangle(self.frame, (ex, ey), (ex+ew, ey+eh), (0,255,0), 2)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [21]:
def viola_jones(img_path):    
    # eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_eye.xml')

    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [24]:
root = Tk()
root.geometry('700x550')
root.title("Face Detectors") 
root.iconbitmap('c:/users/adalv/untitled')

img = Image.open("images/grey.jpg").resize((200, 200), Image.ANTIALIAS)
my_img3 = ImageTk.PhotoImage(img)
img = Image.open("images/grey.jpg").resize((100, 100), Image.ANTIALIAS)
my_img4 = ImageTk.PhotoImage(img)

my_image = Label(image=my_img3)
my_temp = Label(image=my_img4)
my_res = Label(image=my_img3)
my_res1 = Label(image=my_img3)
my_res2 = Label(image=my_img3)
my_res3 = Label(image=my_img3)

def open_img(r):
    global my_image, my_img
    my_image.grid_forget()
    root.filename = filedialog.askopenfilename(initialdir='users/adalv/untitled/images', title="Select A File", filetypes=(("jpg files",".jpg"),("all files","*.*")))
    img = Image.open(root.filename).resize((200, 200), Image.ANTIALIAS)
    my_img = ImageTk.PhotoImage(img)
    my_image = Label(image=my_img)
    if r==1:
        my_image.place(x=50, y=50)
    else:
        my_image.place(x=25, y=50)
        

    
def open_temp(r):
    global my_temp, my_tmp
    root.tempname = filedialog.askopenfilename(initialdir='users/adalv/untitled/images', title="Select A File", filetypes=(("jpg files",".jpg"),("all files","*.*")))
    temp = Image.open(root.tempname).resize((100, 100), Image.ANTIALIAS)
    my_tmp = ImageTk.PhotoImage(temp)
    my_temp = Label(image=my_tmp)
    if r==1:
        my_temp.place(x=300, y=100)
    else:
        my_temp.place(x=25, y=300)
    
    
def results(n):
    global my_res, res
    my_res.destroy()
    if n==1:
        image = Image.fromarray(template_match(root.filename, root.tempname)).resize((200, 200), Image.ANTIALIAS)
        
    elif n==2:
        image = Image.fromarray(viola_jones(root.filename)).resize((200, 200), Image.ANTIALIAS)
        
    elif n==3:
        image = Image.fromarray(symmetry_lines(root.filename)).resize((200, 200), Image.ANTIALIAS)
        
    res = ImageTk.PhotoImage(image)
    my_res = Label(image=res)
    my_res.place(x=450, y=50)
    
    
def results_all():
    global res1, res2, res3, my_res1, my_res2, my_res3
    my_res1.destroy()
    my_res2.destroy()
    my_res3.destroy()
    image1 = Image.fromarray(template_match(root.filename, root.tempname)).resize((200, 200), Image.ANTIALIAS)
    image2 = Image.fromarray(viola_jones(root.filename)).resize((200, 200), Image.ANTIALIAS)
    image3 = Image.fromarray(symmetry_lines(root.filename)).resize((200, 200), Image.ANTIALIAS)
        
    res1 = ImageTk.PhotoImage(image1)
    res2 = ImageTk.PhotoImage(image2)
    res3 = ImageTk.PhotoImage(image3)
    my_res1 = Label(image=res1)
    my_res2 = Label(image=res2)
    my_res3 = Label(image=res3)
    
    my_res1.place(x=250, y=70)
    my_res2.place(x=475, y=70)
    my_res3.place(x=375, y=300)

def refresh(r):
    global flag,flag_all,my_image, my_temp, my_res, btn_temp, btn_mt, btn_vj, btn_sl, btn_all, btn_img, btn_res, my_res1, my_res2, my_res3, lbl_mt, lbl_vj, lbl_sl

    for widget in root.winfo_children():
        widget.destroy()
        
    my_image = Label(image=my_img3)
    my_temp = Label(image=my_img4)
    if r==1:
        my_res = Label(image=my_img3)    
    else: 
        my_res1 = Label(image=my_img3)    
        my_res2 = Label(image=my_img3)    
        my_res3 = Label(image=my_img3) 
        lbl_mt = Label(text='Template Matching')
        lbl_vj = Label(text='Viola-Jones')
        lbl_sl = Label(text='Symmetry Lines')
    flag =True
    
    
def show_all(t, r):
    global my_image, my_temp,my_img, my_tmp, btn_mt, btn_vj, btn_sl, btn_all, btn_img, btn_temp, btn_res, my_res, my_res1, my_res2, my_res3, lbl_mt, lbl_vj, lbl_sl
    btn_mt.pack(side=LEFT, anchor=N)
    btn_vj.pack(side=LEFT, anchor=N)
    btn_sl.pack(side=LEFT, anchor=N)
    btn_all.pack(side=LEFT, anchor=N)
    if r==1:
        my_image.place(x=50, y=50)
        if t==1:
            my_temp.place(x=300, y=100)
            btn_temp.place(x=300, y=320)
        my_res.place(x=450, y=50)

        btn_img.place(x=50, y=320)
        btn_res.place(x=450, y=320)
    else:
        my_image.place(x=25, y=50)
        my_temp.place(x=25, y=300)
        btn_temp.place(x=25, y=420)
        my_res1.place(x=250, y=70)
        my_res2.place(x=475, y=70)
        my_res3.place(x=375, y=300)
        lbl_mt.place(x=250, y=50)
        lbl_vj.place(x=475, y=50)
        lbl_sl.place(x=375, y=280)
        
        
        btn_img.place(x=25, y=270)
        btn_res.place(x=25, y=450)

    
def mt():
    global my_image, my_temp,my_img, my_tmp, btn_mt, btn_vj, btn_sl, btn_all, btn_img, btn_temp, btn_res, my_res 
    refresh(1)
    
    btn_mt = Button(root, text="Template-Matching", state=DISABLED)
    btn_vj = Button(root, text="Viola-Jones", command = lambda: vj())
    btn_sl = Button(root, text="Symmetry lines", command = lambda: sl())
    btn_all = Button(root, text="All tasks", command = lambda: all_tasks())
    btn_img = Button(root, text="Upload image", command = lambda: open_img(1))
    btn_temp = Button(root, text="Upload template", command = lambda: open_temp(1))
    btn_res = Button(root, text="Show results", command = lambda: results(1))
    
    show_all(1,1) 


def vj():
    global my_image, my_temp,my_img, my_tmp, btn_mt, btn_vj, btn_sl, btn_all, btn_img, btn_temp, btn_res, my_res 
    refresh(1)
    
    btn_mt = Button(root, text="Template-Matching", command = lambda: mt())
    btn_vj = Button(root, text="Viola-Jones", state=DISABLED)
    btn_sl = Button(root, text="Symmetry lines", command = lambda: sl())
    btn_all = Button(root, text="All tasks", command = lambda: all_tasks())
    btn_img = Button(root, text="Upload image", command = lambda: open_img(1))
    btn_temp = Button(root, text="Upload template", state=DISABLED)
    btn_res = Button(root, text="Show results", command = lambda: results(2))

    show_all(0,1)
    

def sl():
    global my_image, my_temp,my_img, my_tmp, btn_mt, btn_vj, btn_sl, btn_all, btn_img, btn_temp, btn_res, my_res 
    refresh(1)
    
    btn_mt = Button(root, text="Template-Matching", command = lambda: mt())
    btn_vj = Button(root, text="Viola-Jones", command = lambda: vj())
    btn_sl = Button(root, text="Symmetry lines", state=DISABLED)
    btn_all = Button(root, text="All tasks", command = lambda: all_tasks())
    btn_img = Button(root, text="Upload image", command = lambda: open_img(1))
    btn_temp = Button(root, text="Upload template", state=DISABLED)
    btn_res = Button(root, text="Show results", command = lambda: results(3))
    
    show_all(0,1)
    

def all_tasks():
    global my_image, my_temp,my_img, my_tmp, btn_mt, btn_vj, btn_sl, btn_all, btn_img, btn_temp, btn_res, my_res, my_res1, my_res2, my_res3   
    refresh(3)
    
    btn_mt = Button(root, text="Template-Matching", command = lambda: mt())
    btn_vj = Button(root, text="Viola-Jones", command = lambda: vj())
    btn_sl = Button(root, text="Symmetry lines", command = lambda: sl())
    btn_all = Button(root, text="All tasks", state=DISABLED)
    btn_img = Button(root, text="Upload image", command = lambda: open_img(3))
    btn_temp = Button(root, text="Upload template", command = lambda: open_temp(3))
    btn_res = Button(root, text="Show results", command = lambda: results_all())

    show_all(1,3)
    
    
flag=False
btn_mt = Button(root, text="Template-Matching", command = lambda: mt())
btn_vj = Button(root, text="Viola-Jones", command = lambda: vj())
btn_sl = Button(root, text="Symmetry lines", command = lambda: sl())
btn_all = Button(root, text="All tasks", command = lambda: all_tasks())

btn_mt.pack(side=LEFT, anchor=N)
btn_vj.pack(side=LEFT, anchor=N)
btn_sl.pack(side=LEFT, anchor=N)
btn_all.pack(side=LEFT, anchor=N)

root.mainloop()